In [1]:
from os import environ
environ["AP_PORT"] = "25432" # str(input('port'))
# README command uses port=25432

In [2]:
import pickle
import json
import os
import psycopg2
import numpy as np

from spatialyze.database import Database
from spatialyze.geospatial_video import GeospatialVideo
from spatialyze.road_network import RoadNetwork
from spatialyze.video_processor.camera_config import camera_config
from spatialyze.video_processor.stages.tracking_3d.tracking_3d import Tracking3DResult
from spatialyze.world import World, _execute
from spatialyze.video_processor.cache import disable_cache
from spatialyze.video_processor.metadata_json_encoder import MetadataJSONEncoder

CUDA is available.
 > 0: NVIDIA TITAN Xp
   1: NVIDIA TITAN Xp
   2: NVIDIA TITAN Xp
   3: NVIDIA TITAN Xp
   4: NVIDIA TITAN Xp
   5: NVIDIA TITAN Xp


In [3]:
OUTPUT_DIR = '../../data/pipeline/test-results'
VIDEO_DIR =  '../../data/pipeline/videos'
ROAD_DIR = '../../data/scenic/road-network/boston-seaport'

files = os.listdir(VIDEO_DIR)
with open(os.path.join(VIDEO_DIR, 'frames.pkl'), 'rb') as f:
    videos = pickle.load(f)
    
disable_cache()

In [4]:
database = Database(
    psycopg2.connect(
        dbname=environ.get("AP_DB", "mobilitydb"),
        user=environ.get("AP_USER", "docker"),
        host=environ.get("AP_HOST", "localhost"),
        port=environ.get("AP_PORT", "25432"),
        password=environ.get("AP_PASSWORD", "docker"),
    )
)

In [5]:
world = World(database)
world.addGeogConstructs(RoadNetwork('Boston-Seaport', ROAD_DIR))

In [6]:
for video in videos.values():
    if video['filename'] not in files:
        continue
    
    videofile = os.path.join(VIDEO_DIR, video['filename'])
    camera = [camera_config(*c) for c in video['frames']]

    world.addVideo(GeospatialVideo(videofile, camera))

In [7]:
from spatialyze.utils import F

o = world.object()
c = world.camera()
world.filter((o.type == 'car') &
              F.contained(o.trans@c.time, 'intersection') 
)

In [9]:
objects, trackings = _execute(world)

types ['car']


In [ ]:
from typing import NamedTuple

import numpy as np
import numpy.typing as npt

from ..data_types.camera_config import Float4
from ..video_processor.stages.tracking_3d.tracking_3d import Tracking3DResult, Metadatum as T3DMetadatum


# def get_point(database, filename, frameNum, objectId):
#     point = database.execute(
#         f"""
#         SELECT valueAtTimestamp(t.translations, c.timestamp) 
#         FROM Cameras AS c JOIN Item_General_Trajectory AS t
#         ON c.timestamp <@ t.trajCentroids::period AND c.cameraId =  t.cameraId
#         WHERE c.filename = {filename} AND t.itemId = {objectId} AND c.frameNum = {frameNum}
#         """
#     )
#     return point[0]

def interpolate_track(trackings, objectNum, frameNum):
    left, right = None, None
    leftNum, rightNum = frameNum, frameNum

    while left is None and right is None:
        if left is not None:
            leftNum -= 1
            if objectNum in trackings[video][leftNum]:
                left = trackings[video][leftNum][objectNum]

        if right is not None:
            rightNum += 1
            if objectNum in trackings[video][rightNum]:
                right = trackings[video][rightNum][objectNum]
    
    newPoint = (left.point*leftNum + right.point*rightNum) / (leftNum + rightNum)
    newBboxLeft = (left.bbox_left*leftNum + right.bbox_left*rightNum) / (leftNum + rightNum)
    newBboxTop = (left.bbox_top*leftNum + right.bbox_top*rightNum) / (leftNum + rightNum)
    newBboxHeight = (left.bbox_h*leftNum + right.bbox_h*rightNum) / (leftNum + rightNum)
    newBboxWidth = (left.bbox_w*leftNum + right.bbox_w*rightNum) / (leftNum + rightNum)

    return Tracking3DResult(frame_idx: frameNum, 
                            point: newPoint, 
                            bbox_left: newBboxLeft,
                            bbox_top: newBboxTop, 
                            bbox_h: newBboxHeight,
                            bbox_w: newBboxWidth)

class MovableObject(NamedTuple):
    id: "int"
    type: "str"
    track: "list[npt.NDArray[np.floating]]"
    bboxes: "list[Float4]"
    frame_ids: "list[int]"
    camera_id: "str"


def get_object_list(
    objects: "dict[str, list[tuple]]",
    trackings: "dict[str, list[T3DMetadatum]]",
) -> "list[MovableObject]":
    tracks: "dict[str, dict[int, list[npt.NDArray[np.floating]]]]" = {}
    bboxes: "dict[str, dict[int, list[Float4]]]" = {}
    frameIds: "dict[str, dict[int, list[int]]]" = {}
    objectTypes: "dict[str, dict[int, str]]" = {}

    for video in objects:
        videoObjects = objects[video]
        if len(videoObjects) == 0:
            continue

        cameraId = videoObjects[0][2]  # cameraId same for everything in a video
        assert isinstance(cameraId, str), type(cameraId)
        tracks[cameraId] = {}
        bboxes[cameraId] = {}
        frameIds[cameraId] = {}
        objectTypes[cameraId] = {}

        for obj in videoObjects:
            frameId, objectId =

            objectNum = 

            if objectNum in trackings[frameId]:
                track = trackings[frameId][objectNum]
            else:
                track = interpolate_track(trackings, objectNum, frameId)

            if objectId not in tracks[cameraId]:
                tracks[cameraId][objectId] = []
            tracks[cameraId][objectId].append(track.point)

            if objectId not in bboxes[cameraId]:
                bboxes[cameraId][objectId] = []
            bboxes[cameraId][objectId].append(
                (track.bbox_left, track.bbox_top, track.bbox_w, track.bbox_h)
            )

            if objectId not in frameIds[cameraId]:
                frameIds[cameraId][objectId] = []
            frameIds[cameraId][objectId].append(track.frame_idx)
            
            objectTypes[cameraId][objectId] = track.object_type

    result: "list[MovableObject]" = []
    for cameraId in tracks:
        for objectId in tracks[cameraId]:
            result.append(
                MovableObject(
                    objectId,
                    objectTypes[cameraId][objectId],
                    tracks[cameraId][objectId],
                    bboxes[cameraId][objectId],
                    frameIds[cameraId][objectId],
                    cameraId,
                )
            )
    return result

In [ ]:
import cv2

from ..video_processor.stages.tracking_3d.tracking_3d import Metadatum as T3DMetadatum
from ..video_processor.stages.tracking_3d.tracking_3d import Tracking3DResult


def save_video_util(
    objects: "dict[str, list[tuple]]",
    trackings: "dict[str, list[T3DMetadatum]]",
    OUTPUT_PATH: "str",
    addBoundingBoxes: "bool" = False,
) -> "list[tuple[str, int]]":
    frame_trackings = _get_frame_objects(trackings)
    cameraIds = _get_camera_ids(objects)

    result: "list[tuple[str, int]]" = []

    for videoname, frame_tracking in frame_trackings.items():
        cameraId = cameraIds[videoname]
        output_file = OUTPUT_PATH + cameraId + "-result.mp4"

        cap = cv2.VideoCapture(videoname)
        if not cap.isOpened():
            print(f"WARNING: Cannot read video file: {videoname}")
            continue

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        vid_writer = cv2.VideoWriter(
            output_file, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height)
        )

        frame_cnt = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_cnt in frame_tracking:
                if addBoundingBoxes:
                    for track in frame_tracking.get(frame_cnt, []):
                        bbox_left, bbox_top, bbox_w, bbox_h = (
                            track.bbox_left,
                            track.bbox_top,
                            track.bbox_w,
                            track.bbox_h,
                        )
                        x1, y1 = bbox_left, bbox_top
                        x2, y2 = bbox_left + bbox_w, bbox_top - bbox_h
                        frame = cv2.rectangle(
                            frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 0), 2
                        )

                vid_writer.write(frame)
                result.append((videoname, frame_cnt))

            frame_cnt += 1

        vid_writer.release()

    return result
    


def _get_frame_objects(trackings: "dict[str, list[T3DMetadatum]]"):
    """
    Indexes objects based on frame ID
    """
    result: "dict[str, dict[int, list[Tracking3DResult]]]" = {}
    for video, tracking in trackings.items():
        result[video] = {}
        for frame in tracking:
            for objectId in frame:
                track = frame[objectId]
                frameId = track.frame_idx
                if frameId not in result[video]:
                    result[video][frameId] = []
                result[video][frameId].append(track)

    return result


def _get_camera_ids(objects: "dict[str, list[tuple]]"):
    """
    Gets the cameraIds relating to each of the videos
    """
    result: "dict[str, str]" = {}
    for video, obj in objects.items():
        if len(obj) == 0:
            continue

        result[video] = obj[0][2]
    return result

In [ ]:
trackings['../../data/pipeline/videos\\boston-seaport-scene-0655-CAM_FRONT.mp4'][8]

In [ ]:
objects